# Packages

In [1]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.distributed import Client
from folders import *
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import osmnx as ox
import pandas as pd
from tqdm.auto import tqdm

# Directory

In [2]:
path = r"C:\Users\jigon\OneDrive\Documentos\Economía\Commuting-Zones-Costa-Rica"
os.chdir(path)

# Dask setup

In [3]:
ProgressBar().register()
N_THREADS = 10 # leave at least 3GB of RAM to each workers. 32GB/10 = 3.2GB OK
client = Client(n_workers=N_THREADS, threads_per_worker=1)  # Connect to distributed cluster and override default
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 10,Total memory: 31.71 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49237,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 31.71 GiB
Comm: tcp://127.0.0.1:49279,Total threads: 1
Dashboard: http://127.0.0.1:49280/status,Memory: 3.17 GiB
Nanny: tcp://127.0.0.1:49249,


# Read network dataset

In [4]:
G = ox.graph_from_xml(costa_rican_roads_file, simplify=True, retain_all=True)
costa_rica_map = gpd.read_file(costa_rica_shapefile)
municipalities_coordinates = pd.read_excel(costa_rican_commuting_zones_file).sort_values("municipality")
municipalities_area = pd.read_excel(municipalities_area_file).sort_values("municipality")

# Recode municipalities

In [5]:
municipalities_area["municipality"] = municipalities_area["municipality"].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Filter graph to retain only certain edge types

In [6]:
filtr = ['primary', 'secondary', 'tertiary', 'trunk', 'residential']
e = [(u, v, k) for u, v, k, d in G.edges(keys=True, data=True) if 'highway' not in d.keys()]
G.remove_edges_from(e)
e = [(u, v, k) for u, v, k, d in G.edges(keys=True, data=True) if d['highway'] not in filtr]
G.remove_edges_from(e)

# Remove any now-disconnected nodes or subcomponents, then simplify topology

In [7]:
G = ox.utils_graph.get_largest_component(G)

# Add speed values


In [8]:
G = ox.add_edge_speeds(G)
nodes, edges = ox.graph_to_gdfs(G)

# Compute average speed by municipality to compute distance of a municipality to itself

In [9]:
streets = edges.reset_index(drop=True)[["geometry", "speed_kph", "length"]].copy()
municipalities_borders = costa_rica_map[["geometry", "ADM2_ES"]].copy()
municipalities_borders.rename(columns={"ADM2_ES": "municipality"}, inplace=True)
municipalities_borders["municipality"] = municipalities_borders["municipality"].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
avg_speed_municipalities = gpd.sjoin(streets, municipalities_borders, how="left")
avg_speed_municipalities = avg_speed_municipalities.drop(columns=["geometry", "index_right"])

In [10]:
municipalities_borders = costa_rica_map[["geometry", "ADM2_ES"]].copy()
municipalities_borders.rename(columns={"ADM2_ES": "municipality"}, inplace=True)
municipalities_borders["municipality"] = municipalities_borders["municipality"].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

## Compute average by municipality

In [11]:
avg_speed_municipalities["avg_speed"] = avg_speed_municipalities["speed_kph"] * avg_speed_municipalities["length"]
avg_speed_municipalities = avg_speed_municipalities.groupby("municipality")[["avg_speed", "length"]].sum()
avg_speed_municipalities["avg_speed"] /= avg_speed_municipalities["length"]
avg_speed_municipalities = avg_speed_municipalities.reset_index()
avg_speed_municipalities.drop(columns=["length"], inplace=True)

# Imputate missing values

In [12]:
edges["speed_kph"] = edges["speed_kph"].fillna(edges["speed_kph"].mean())

# Impute edge (driving) speeds and calculate edge traversal times

In [13]:
nx.set_edge_attributes(G, values=edges["speed_kph"], name="speed_kph")
G = ox.add_edge_travel_times(G)

# Find nearest node for each municipality office
Remember we're using their offices as a reference point for the whole municipality. 

In [14]:
municipalities_nodes = ox.distance.nearest_nodes(G, municipalities_coordinates["lon"], municipalities_coordinates["lat"])
municipalities_coordinates["node"] = municipalities_nodes

# Calculate distance matrix based on travel time

## Create cartesian product of municipalities

In [15]:
municipalities_coordinates = municipalities_coordinates.merge(municipalities_coordinates, how="cross", suffixes=('_origin', '_destination'))
municipalities_coordinates = dd.from_pandas(municipalities_coordinates, npartitions=N_THREADS)

## Compute route distance and travel time through shortest path

- **Travel time:** minutes
- **Distance:** kilometers

## Functions

In [16]:
def shortest_path_dask(dds, col_1, col_2): 
    return ox.distance.shortest_path(G, dds[col_1], dds[col_2], weight="travel_time")
def travel_time(df): 
    return sum(ox.utils_graph.get_route_edge_attributes(G, df['path'], "travel_time")) / 60
def distance(df): 
    return sum(ox.utils_graph.get_route_edge_attributes(G, df['path'], "length")) / 1_000

## Computation
- **Speed:** Kilometers per hour. 
- **Route distance:** Kilometers. 
- **Travel time:** Minutes. 

In [17]:
municipalities_coordinates["path"] = municipalities_coordinates.apply(shortest_path_dask, axis=1, args=("node_origin", "node_destination"), meta=("path", "object"))
with ProgressBar():
    municipalities_coordinates = municipalities_coordinates.compute()
tqdm.pandas(desc="Travel time")
municipalities_coordinates['travel_time'] = municipalities_coordinates.progress_apply(travel_time, axis=1)
tqdm.pandas(desc="Distance")
municipalities_coordinates['distance'] = municipalities_coordinates.progress_apply(distance, axis=1)
municipalities_coordinates["avg_speed"] = 60 * municipalities_coordinates['distance'] / municipalities_coordinates['travel_time']

Travel time:   0%|          | 0/6561 [00:00<?, ?it/s]

Distance:   0%|          | 0/6561 [00:00<?, ?it/s]

# Neighbor dummies

In [18]:
municipalities_borders = municipalities_borders.merge(municipalities_borders, how="cross", suffixes=('_origin', '_destination'))
municipalities_borders["neighbor"] = municipalities_borders.apply(lambda x: ~x["geometry_origin"].disjoint(x["geometry_destination"]), axis=1)
municipalities_borders.loc[(municipalities_borders["municipality_origin"] == municipalities_borders["municipality_destination"]), "neighbor"] = False
municipalities_borders.loc[(municipalities_borders["municipality_origin"] == "canas") | (municipalities_borders["municipality_destination"] == "canas"), "neighbor"] = False

# Manually Fix Canas problem
municipalities_borders.loc[(municipalities_borders["municipality_origin"] == "canas") & (municipalities_borders["municipality_destination"] == "guatuso"), "neighbor"] = True
municipalities_borders.loc[(municipalities_borders["municipality_origin"] == "canas") & (municipalities_borders["municipality_destination"] == "bagaces"), "neighbor"] = True
municipalities_borders.loc[(municipalities_borders["municipality_origin"] == "canas") & (municipalities_borders["municipality_destination"] == "tilaran"), "neighbor"] = True
municipalities_borders.loc[(municipalities_borders["municipality_origin"] == "canas") & (municipalities_borders["municipality_destination"] == "abangares"), "neighbor"] = True
municipalities_borders.loc[(municipalities_borders["municipality_origin"] == "canas") & (municipalities_borders["municipality_destination"] == "nicoya"), "neighbor"] = True
municipalities_borders.loc[(municipalities_borders["municipality_destination"] == "canas") & (municipalities_borders["municipality_origin"] == "guatuso"), "neighbor"] = True
municipalities_borders.loc[(municipalities_borders["municipality_destination"] == "canas") & (municipalities_borders["municipality_origin"] == "bagaces"), "neighbor"] = True
municipalities_borders.loc[(municipalities_borders["municipality_destination"] == "canas") & (municipalities_borders["municipality_origin"] == "tilaran"), "neighbor"] = True
municipalities_borders.loc[(municipalities_borders["municipality_destination"] == "canas") & (municipalities_borders["municipality_origin"] == "abangares"), "neighbor"] = True
municipalities_borders.loc[(municipalities_borders["municipality_destination"] == "canas") & (municipalities_borders["municipality_origin"] == "nicoya"), "neighbor"] = True
municipalities_borders["neighbor"] = municipalities_borders["neighbor"].astype("bool")

municipalities_borders.drop(columns=["geometry_origin", "geometry_destination"], inplace=True)
municipalities_borders.loc[(municipalities_borders["municipality_origin"] == "canas"), "municipality_origin"] = "cañas"
municipalities_borders.loc[(municipalities_borders["municipality_destination"] == "canas"), "municipality_destination"] = "cañas"
municipalities_coordinates = municipalities_coordinates.merge(municipalities_borders, on=["municipality_origin", "municipality_destination"], how="left")

# Interstate dummies

In [19]:
municipalities_coordinates["interstate"] = municipalities_coordinates["province_origin"] != municipalities_coordinates["province_destination"]

# Own municipality distance

In [20]:
avg_speed_municipalities["municipality_destination"] = avg_speed_municipalities["municipality"]
avg_speed_municipalities.rename(columns={"municipality": "municipality_origin"}, inplace=True)
municipalities_area["municipality_destination"] = municipalities_area["municipality"]
municipalities_area.rename(columns={"municipality": "municipality_origin"}, inplace=True)
avg_speed_municipalities = avg_speed_municipalities.merge(municipalities_area, on=["municipality_origin", "municipality_destination"], how="left")
avg_speed_municipalities["distance"] = (2 / 3) * np.sqrt(avg_speed_municipalities["area"] * np.pi)
avg_speed_municipalities["travel_time"] = 60 * avg_speed_municipalities["distance"] / avg_speed_municipalities["avg_speed"]
avg_speed_municipalities.drop(columns=["area"], inplace=True)
avg_speed_municipalities.rename(columns={"distance": "distance_own", "travel_time": "travel_time_own", "avg_speed": "avg_speed_own"}, inplace=True)
municipalities_coordinates = municipalities_coordinates.merge(avg_speed_municipalities, on=["municipality_origin", "municipality_destination"], how="left")
municipalities_coordinates.loc[(municipalities_coordinates["distance"] == 0), "distance"] = np.nan
municipalities_coordinates.loc[(municipalities_coordinates["travel_time"] == 0), "travel_time"] = np.nan
municipalities_coordinates.loc[(municipalities_coordinates["avg_speed"] == 0), "avg_speed"] = np.nan
municipalities_coordinates["distance"].fillna(municipalities_coordinates["distance_own"], inplace=True)
municipalities_coordinates["travel_time"].fillna(municipalities_coordinates["travel_time_own"], inplace=True)
municipalities_coordinates["avg_speed"].fillna(municipalities_coordinates["avg_speed_own"], inplace=True)
municipalities_coordinates.drop(columns=["distance_own", "travel_time_own", "avg_speed_own"], inplace=True)

# Keep variables of interest

In [21]:
municipalities_coordinates = municipalities_coordinates[['municipality_origin', 'CZ_origin', 'region_origin', 'province_origin', 'municipality_destination', 
                                                         'CZ_destination', 'region_destination', 'province_destination', 'employment_origin', 'employment_destination', 
                                                         'travel_time', 'distance', 'avg_speed', 'neighbor', 'interstate']].copy()

# Save distance matrix

In [22]:
municipalities_coordinates.to_excel(costa_rican_municipalities_distance_matrix, index=False)
(municipalities_coordinates[["municipality_origin", "municipality_destination", "travel_time", "distance", "neighbor", 
                             "interstate"]].rename(columns={"municipality_origin": "s_canton", 
                                                            "municipality_destination": "b_canton", 
                                                            "travel_time": "time"})
                                           .to_stata(costa_rican_municipalities_distance_matrix_stata, write_index=False))